In [1]:
import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np
from lmfit import  Model, Parameters

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [3]:
equation='fct_velocity'
time_sup = 280
opt_base = {'stop_search_misac': None, 'time_sup': 280,
            'param_fit': None, 'inde_vars': None,
            'step_fit':2, 'do_whitening': False,
            'before_sacc': 5, 'after_sacc': 15}    

In [4]:
x = 11
timeStr = time[x]
observer = sujet[x]
mode = 'enregistrement'

print(observer,timeStr)

file = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.pkl')
with open(file, 'rb') as fichier :
    param_exp = pickle.load(fichier, encoding='latin1')

resultats = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.asc')
data = read_edf(resultats, 'TRIALID')

A = ANEMO(param_exp)
Fit = ANEMO.Fit(param_exp)

YK 2017-11-17_172706


In [5]:
N_trials = param_exp['N_trials']
N_blocks = param_exp['N_blocks']

goodness_of_fit = {}

for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
    goodness_of_fit[s] = []

for block in range(N_blocks) :
    
    for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
        goodness_of_fit[s].append([])
    
    for trial in range(N_trials) :

        print(block, trial)

        trial_data = trial + N_trials*block
        arg = A.arg(data[trial_data], trial=trial, block=block)
        opt = opt_base.copy()
        opt.update(arg)

        velocity_NAN = A.velocity_NAN(**opt)

        #-------------------------------------------------
        # FIT
        #-------------------------------------------------
        if equation=='fct_velocity' :
            data_trial = velocity_NAN
        else :
            data_trial = A.data_deg(data=arg.data_x, **opt)

        old_latence, old_max, old_anti = A.classical_method.Full(velocity_NAN, arg.TargetOn-arg.t_0)
        f = Fit.Fit_trial(data_trial, equation=equation, value_latence=old_latence, value_max=old_max, value_anti=old_anti, **opt)
        #-------------------------------------------------
        goodness_of_fit['nfev'][block].append(f.nfev)
        goodness_of_fit['residual'][block].append(f.residual)
        goodness_of_fit['chisqr'][block].append(f.chisqr)
        goodness_of_fit['redchi'][block].append(f.redchi)
        goodness_of_fit['aic'][block].append(f.aic)
        goodness_of_fit['bic'][block].append(f.bic)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72


/usr/local/lib/python3.6/dist-packages/ANEMO-0.0.1-py3.6.egg/ANEMO/ANEMO.py:574: RuntimeWarning: Mean of empty slice


0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177


/usr/local/lib/python3.6/dist-packages/ANEMO-0.0.1-py3.6.egg/ANEMO/ANEMO.py:376: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ANEMO-0.0.1-py3.6.egg/ANEMO/ANEMO.py:376: RuntimeWarning: invalid value encountered in true_divide


0 178
0 179
0 180
0 181
0 182
0 183
0 184


/usr/local/lib/python3.6/dist-packages/ANEMO-0.0.1-py3.6.egg/ANEMO/ANEMO.py:594: RuntimeWarning: Mean of empty slice


0 185
0 186
0 187
0 188
0 189
0 190
0 191
0 192
0 193
0 194
0 195
0 196
0 197
0 198
0 199
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169


In [6]:
file = os.path.join('parametre', 'goodness_of_fit_%s.pkl'%(observer))
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(goodness_of_fit)

In [7]:
goodness_of_fit

{'nfev': [[680,
   488,
   307,
   532,
   384,
   484,
   1421,
   547,
   678,
   425,
   531,
   459,
   630,
   399,
   1166,
   574,
   397,
   527,
   731,
   699,
   302,
   263,
   377,
   416,
   487,
   684,
   581,
   569,
   871,
   491,
   433,
   641,
   433,
   370,
   563,
   388,
   257,
   704,
   615,
   542,
   363,
   468,
   508,
   557,
   674,
   490,
   465,
   882,
   490,
   519,
   501,
   634,
   563,
   1039,
   633,
   601,
   701,
   379,
   483,
   486,
   245,
   453,
   377,
   554,
   570,
   360,
   260,
   489,
   917,
   531,
   599,
   369,
   395,
   354,
   894,
   475,
   755,
   512,
   521,
   917,
   934,
   773,
   507,
   881,
   896,
   703,
   502,
   1011,
   419,
   426,
   704,
   819,
   1039,
   1537,
   772,
   585,
   318,
   484,
   435,
   496,
   601,
   702,
   504,
   342,
   653,
   474,
   642,
   406,
   472,
   642,
   443,
   879,
   371,
   725,
   318,
   528,
   843,
   994,
   325,
   527,
   273,
   429,
   462,
  